In [1]:
import pandas as pd 
import numpy as np
from pandas import DataFrame, Series
import csv

In [2]:
#Importing Earthquake Data
eq_data = pd.read_csv('../Files/processed-data/completeEarthquakeFreq.csv')
eq_data['origintime'] = pd.to_datetime(eq_data['origintime'])
eq_data['YEAR'] =  eq_data.origintime.apply(lambda x: x.year)
eq_data = eq_data[eq_data['YEAR'] != 2016]
eq_cumulative = eq_data.groupby(['county','YEAR'],as_index=False)['id'].sum()
eq_cumulative['eq_count'] = eq_data.groupby(['county','YEAR'],as_index=False).size().values
eq_cumulative = eq_cumulative.drop('id', 1)

#Import Production data
prod = pd.read_csv('../Files/processed-data/prod_summ.csv').\
ix[:,["API","WELLID","FIRSTPRODDATE","LASTPRODDATE","CUMULATIVEOIL","FIRSTTWELVEMONTHOIL","FIRSTTWENTYFOURMONTHOIL"]]

well_header = pd.read_csv("../Files/processed-data/well_header.csv").\
ix[:,["WELLID","COUNTY","SURFACELATITUDE","SURFACELONGITUDE","WELLBOREPROFILE"]]

county = pd.read_csv("../Files/processed-data/county.csv").ix[:,["Geo_ID","State","COUNTY"]]
combined_data = pd.merge(pd.merge(prod,well_header),county)
combined_data.FIRSTPRODDATE = pd.to_datetime(combined_data.FIRSTPRODDATE)
combined_data.LASTPRODDATE = pd.to_datetime(combined_data.LASTPRODDATE)
combined_data = combined_data[combined_data.WELLBOREPROFILE == "HORIZONTAL"]
combined_data['Start_Year'] = combined_data.FIRSTPRODDATE.dt.year
combined_data['End_Year'] = combined_data.LASTPRODDATE.dt.year


C:\Users\Anirudh Narayanan\AppData\Local\Enthought\Canopy\User\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Approximating Oil Production by year

In [3]:
for i in range(1978, 2016):
    combined_data[i] = np.array((combined_data['End_Year'] >= i) & (combined_data['Start_Year'] <= i)).astype(int)

combined_data['SECONDTWELVEMONTHOIL'] = combined_data['FIRSTTWENTYFOURMONTHOIL'] - combined_data['FIRSTTWELVEMONTHOIL']
combined_data['cum_minus_2_year'] = combined_data['CUMULATIVEOIL'] - combined_data['FIRSTTWENTYFOURMONTHOIL']

def prod_approximation(x):
    a = list(x).index(1)
    x[a] = x.FIRSTTWELVEMONTHOIL
    x[a+1] = x.SECONDTWELVEMONTHOIL
    x[x == 1] = x.cum_minus_2_year / sum(x == 1)
    return x

data_approx = list()
for i in range(combined_data.shape[0]):
    data_approx.append(prod_approximation(combined_data.ix[i,]).values)
    
data_approx = DataFrame(data_approx)
data_approx.columns = combined_data.columns

data_well_cum = data_approx.groupby('COUNTY')[range(1978,2016)].sum().round(2)
data_well_cum['county'] = data_well_cum.index
data_well_cum = pd.melt(data_well_cum, id_vars=["county"], var_name="YEAR", value_name="Cum_prod")

KeyError: 0L

## Preparing data for regression

In [ ]:
eq_cumulative.to_csv("C:/Users/Aayush - Carlson/Downloads/Python/eq_cumulative.csv",header=True, index=False)
data_well_cum.to_csv("C:/Users/Aayush - Carlson/Downloads/Python/data_well_cum.csv",header=True, index=False)

analysis_data = pd.merge(data_well_cum,eq_cumulative,on=['county','YEAR'],how="left").fillna(0)
analysis_data1 = analysis_data[analysis_data.YEAR>=2005]
analysis_data1.sort_values(by = ['county','YEAR'],inplace=True)
analysis_data1.index = range(0,len(analysis_data1.index))
analysis_data1['production_lag_2years'] = analysis_data1.Cum_prod.shift(2)[analysis_data1.county == analysis_data1.county.shift(2)]
analysis_data1.head()

## Modelling

In [ ]:
import numpy as np
import statsmodels.formula.api as smf
mod = smf.ols("eq_count ~ np.log(production_lag_2years+1)", analysis_data1.ix[11:].dropna()).fit()
print(mod.summary())